In [1]:
import numpy as np
from scipy import linalg
import time

In [2]:
from numba import jit

@jit(nopython=True)
def forwardElimination(L,b):
    x = np.zeros(b.shape)
    for index in range(L.shape[0]):
        x[index] = (b[index] - np.sum(L[index][:index]*x[:index]))/L[index][index]
    return x

@jit(nopython=True)
def backwardElimination(U,b):
    return forwardElimination(U[::-1,::-1],b[::-1])[::-1]

L = np.tril(np.random.randn(10,10))
with np.printoptions(precision = 2,suppress=True):
    print('L')
    print(L)
b = np.random.randn(10)

print()

x = linalg.solve(L,b)
xtest = forwardElimination(L,b)
print('Forward elimination working?  {}'.format(np.allclose(x,xtest)))

x = linalg.solve(L.T,b)
xtest = backwardElimination(L.T,b)
print('Backward elimination working? {}'.format(np.allclose(x,xtest)))


L
[[ 0.54  0.    0.    0.    0.    0.    0.    0.    0.    0.  ]
 [-0.26 -1.62  0.    0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.83  0.22 -1.5   0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.94  0.54 -0.92  0.09  0.    0.    0.    0.    0.    0.  ]
 [ 1.87 -1.36  1.3  -0.58  0.43  0.    0.    0.    0.    0.  ]
 [ 0.31  2.36 -0.81 -1.09 -0.27 -0.15  0.    0.    0.    0.  ]
 [-0.07 -1.39  0.84  0.76  0.42  0.61 -0.29  0.    0.    0.  ]
 [-0.4   0.   -0.27 -1.33 -0.88 -0.98 -0.49 -0.07  0.    0.  ]
 [-0.41 -2.47  1.33 -0.6   1.43 -0.93  0.26  0.33  1.48  0.  ]
 [-0.34 -0.39  0.29 -0.61  0.31  0.99 -0.03 -0.39 -0.17  0.52]]

Forward elimination working?  True
Backward elimination working? True


In [3]:
n = 3000

A = np.random.randn(n,n)
b = np.random.randn(n)

In [4]:
%%time
x = linalg.solve(A, b)

CPU times: user 1.13 s, sys: 45.9 ms, total: 1.17 s
Wall time: 396 ms


In [5]:
%%time 
P, L, U = linalg.lu(A)

CPU times: user 719 ms, sys: 50 ms, total: 769 ms
Wall time: 334 ms


In [6]:
print(np.allclose(A,P@L@U))

True


Permute right-hand side

In [7]:
bperm = P.T@b

In [9]:
%%time
xLU = forwardElimination(L,bperm)
xLU = backwardElimination(U,xLU)
print(np.allclose(x,xLU))

True
CPU times: user 61.7 ms, sys: 1.62 ms, total: 63.3 ms
Wall time: 62.7 ms


In [10]:
%%time 
Q, R = linalg.qr(A)

CPU times: user 3.03 s, sys: 66.1 ms, total: 3.1 s
Wall time: 965 ms


In [12]:
%%time
xQR = backwardElimination(R,Q.T@b)
print(np.allclose(x,xQR))

True
CPU times: user 58.1 ms, sys: 3.06 ms, total: 61.1 ms
Wall time: 18.7 ms


In [13]:
%%time
U, S, V = linalg.svd(A)
V = V.T

CPU times: user 20.7 s, sys: 127 ms, total: 20.9 s
Wall time: 5.28 s


In [14]:
%%time 
xSVD = V @(U.T@b/S)
print(np.allclose(x,xSVD))

True
CPU times: user 28.9 ms, sys: 2.17 ms, total: 31 ms
Wall time: 7.61 ms


In [15]:
S.shape

(3000,)

In [16]:
S[:10]

array([109.40547274, 109.02973681, 108.89984915, 108.53554136,
       108.32159236, 108.20310186, 108.00111541, 107.71624539,
       107.50154436, 107.41184628])